In [6]:
# 손실함수  
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0) # 오버플로를 막기 위해서 exp안에 최댓값을 빼줌 
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로를 막기 위해서 exp안에 최댓값을 빼줌 
    return np.exp(x) / np.sum(np.exp(x))


# 배치용 교차 엔트로피 오차 
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    delta = 1e-7  # 1e-7은 log안에 0이 되지 않도록 더해주는 값
    
    # 배치의 크기로 나눠 정규화 하고 평균 교차 엔트로피 오차를 계산 
    return -np.sum(np.log(y[np.arange(batch_size), t] + delta)) / batch_size

def sigmoid(x) :
    return 1 / (1+np.exp(-x))

In [7]:
# 활성화 함수 

class Relu:
    def __init__(self):
        self.mask = None # mask : T/F로 구성된 배열

    def forward(self, x):
        self.mask = (x <= 0) # 0보다 같거나 작으면 True(1)
        out = x.copy()
        out[self.mask] = 0 # fancy index : out[x<=0] = 0 -> 0보다 작은 원소들을 0으로 

        return out

    def backward(self, dout): 
        dout[self.mask] = 0   # 역전파시 0보다 작은 원소들 0으로    
        dx = dout

        return dx

# sigmoid : y = 1 / (1+exp(-x))
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out   # 역전파 시 y(1-y) 곱해줌

        return dx


# Wx + b
# 신경망의 순전파 때 수행하는 행렬의 곱을 기하학에서 affine transformation이라고 함 
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경
        return dx


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

In [8]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
# from common.layers import *
# from common.gradient import numerical_gradient
from collections import OrderedDict



class MLP:

    def __init__(self, input_size, hidden_size_1,hidden_size_2, output_size, weight_init_std = 0.01):
        
        # 층 3개  
        self.params = {} # 딕셔너리 변수
        
        # 1번째 층의 가중치와 편향
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size_1)
        self.params['b1'] = np.zeros(hidden_size_1)
        # 2번쨰 층의 가중치와 편향
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size_1, hidden_size_2) 
        self.params['b2'] = np.zeros(hidden_size_2)
        # 3번째 층의 가중치와 편향
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size_2, output_size)
        self.params['b3'] = np.zeros(output_size)
        
      
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
      #  self.layers['Sigmoid1'] = Sigmoid()
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
      #  self.layers['Sigmoid2'] = Sigmoid()
        self.layers['Relu2'] = Relu()
        self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])
        
        self.lastLayer = SoftmaxWithLoss()
    
    # 예측 함수 
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
        
    # x : 입력 데이터, t : 정답 레이블
    # 손실함수의 값을 구함 
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t) # SoftmaxWithLoss y,t의 cross_entropy
    
    # 정확도 
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
#     def numerical_gradient(self, x, t):
#         loss_W = lambda W: self.loss(x, t)
        
#         grads = {}
#         grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
#         grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
#         grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
#         grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
#         return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1                             # 1로 초기화 
        dout = self.lastLayer.backward(dout) # lastLayer : softmax 부터 
        
        layers = list(self.layers.values()) # layer들의 리스트 
        layers.reverse()
        for layer in layers:                  # 역전파 
            dout = layer.backward(dout) 

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W3'], grads['b3'] = self.layers['Affine3'].dW, self.layers['Affine3'].db
        
        return grads

In [11]:
# 입력층 784, 출력층 10 
from data.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)
print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

(60000, 784) (60000, 10)
(10000, 784) (10000, 10)


In [64]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)

import numpy as np
from data.mnist import load_mnist

# 데이터 mnist데이터 셋에서 onehot데이터 load
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = MLP(input_size=784, hidden_size_1=50, hidden_size_2 = 25,output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)   # train_size = 60000, batch_size = 100, 60000개 중 임의 100개 
    x_batch = x_train[batch_mask]   
    t_batch = t_train[batch_mask] 
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch) 
    train_loss_list.append(loss) 
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train accuracy:", train_acc,"test accuracy:", test_acc)

train accuracy: 0.10218333333333333 test accuracy: 0.101
train accuracy: 0.4860333333333333 test accuracy: 0.4879
train accuracy: 0.8671 test accuracy: 0.8727
train accuracy: 0.9166333333333333 test accuracy: 0.9151
train accuracy: 0.93705 test accuracy: 0.934
train accuracy: 0.9472833333333334 test accuracy: 0.9456
train accuracy: 0.95495 test accuracy: 0.9519
train accuracy: 0.96425 test accuracy: 0.9581
train accuracy: 0.9664166666666667 test accuracy: 0.9609
train accuracy: 0.9686166666666667 test accuracy: 0.9616
train accuracy: 0.9732166666666666 test accuracy: 0.9659
train accuracy: 0.97435 test accuracy: 0.966
train accuracy: 0.97755 test accuracy: 0.9688
train accuracy: 0.9775166666666667 test accuracy: 0.9675
train accuracy: 0.9807 test accuracy: 0.968
train accuracy: 0.9837166666666667 test accuracy: 0.9716
train accuracy: 0.9835 test accuracy: 0.9681


In [20]:
train_size = x_train.shape[0]
batch_size = 100
np.random.choice(train_size, batch_size)
train_size

60000

In [63]:
np.random.choice(5,1)

array([1])

--------------------

In [97]:
class MLP2 : 
    """
    parameters
    input_size : 입력크기
    hidden_size_list : 은닉층의 뉴런 수 리스트
    output_size : 출력크기 
    activation : relu / sigmoid  
    weight_init_std : 가중치의 표준편차 지정
    weight_decay_lambda : 가중치 감소의 세기
    """
    
    def __init__(self, input_size, hidden_size_list, output_size, activation ='relu', weight_init_std = 'relu', 
                 weight_decay_lambda=0) :
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size_list = hidden_size_list
        self.hidden_layer_num = len(hidden_size_list)
        self.weight_decay_lambda = weight_decay_lambda
        self.params = {}
        
        # 가중치 초기화
        self.__init_weight(weight_init_std)
        
        # 계층 생성
        activation_layer = {'sigmoid' : Sigmoid, 'relu' : Relu}
        self.layers = OrderedDict()
        for idx in range(1, self.hidden_layer_num+1) :
            self.layers['Affine' + str(idx)] = Affine(self.params['W'+str(idx)], self.params['b'+str(idx)])
            self.layers['Activation_function' + str(idx)] = activation_layer[activation]()
        
        idx = self.hidden_layer_num + 1
        # 어파인층 개수 : 은닉층개수 + 1 
        self.layers['Affine'+str(idx)] = Affine(self.params['W'+str(idx)], self.params['b' + str(idx)])
        
        self.last_layer = SoftmaxWithLoss()
    
    def __init_weight(self, weight_init_std) :
        """가중치 초기화 
        Parameters
        ----------
        weight_init_std : 가중치의 표준편차
            relu, he로 지정 -> He 초깃값
            sigmoid, xavier로 지정 -> Xavier 초깃값
        """
        all_size_list = [self.input_size] + self.hidden_size_list + [self.output_size]
        for idx in range(1, len(all_size_list)) :
            scale = weight_init_std
            if str(weight_init_std).lower() in ('relu', 'he') :
                scale = np.sqrt(2.0 / all_size_list[idx - 1]) # ReLU를 사용할 때의 권장 초깃값 루트 2/n
            elif str(weight_init_std).lower() in ('sigmoid', 'xavier') :
                scale = np.sqrt(1.0 / all_size_list[idx - 1]) # sigmoid를 사용할 때의 권장 초깃값 루트 1/n
            self.params['W'+ str(idx)] = scale*np.random.randn(all_size_list[idx-1], all_size_list[idx])
            self.params['b'+ str(idx)] = np.zeros(all_size_list[idx])
         
        
    def predict(self, x) :
        for layer in self.layers.values() :
            x = layer.forward(x)
        
        return x
    
    def loss(self, x, t) :
        """
        손실 함수
        parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        """
        y = self.predict(x)
        
        weight_decay = 0
        for idx in range(1, self.hidden_layer_num + 2) :
            W = self.params['W' + str(idx)]
            weight_decay += 0.5*self.weight_decay_lambda * np.sum(W**2)
        
        return self.last_layer.forward(y,t) + weight_decay
    
    def accuracy(self, x, t) :
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
#     def numerical_gradient(self, x, t) :
#         loss_W = lambda W : self.loss(x,t)
        
#         grads = {}
#         for idx in range(1, self.hidden_layer_num + 2) :
#             grads['W' + str(idx)] = numerical_gradient(loss_W, self.params['W' + str(idx)])
#             grads['b' + str(idx)] = numerical_gradient(loss_W, self.params['b' + str(idx)])
        
#         return grads
    
    def gradient(self, x, t) :
        # forward
        self.loss(x, t)
        
        # backward
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers :
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        for idx in range(1, self.hidden_layer_num + 2) :
            # W에는 가중치 감소 
            grads['W' + str(idx)] = self.layers['Affine'+ str(idx)].dW + self.weight_decay_lambda * self.layers['Affine' +str(idx)].W 
            grads['b' + str(idx)] = self.layers['Affine'+ str(idx)].db
        
        return grads
        
        

In [98]:
class SGD:
    """확률적 경사 하강법（Stochastic Gradient Descent）"""

    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key] 


In [99]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

In [100]:
network = MLP2(input_size=784, hidden_size_list =[250,50], output_size = 10)

In [102]:
network = MLP2(input_size = 784, hidden_size_list = [100, 100, 100, 100, 100, 100], output_size = 10)
optimizer = SGD(lr = 0.01) # 학습률이 0.01인 SGD로 매개변수 갱신
max_epochs = 201
train_size = x_train.shape[0]
batch_size = 100

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
epoch_cnt = 0

for i in range(50000) :
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grads = network.gradient(x_batch ,t_batch)
    optimizer.update(network.params, grads)
    
    if i%iter_per_epoch == 0 :
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train accuracy:", train_acc,"test accuracy:", test_acc)
        
        epoch_cnt += 1
        if epoch_cnt >= max_epochs :
            break


train accuracy: 0.1082 test accuracy: 0.1144
train accuracy: 0.8944833333333333 test accuracy: 0.8969
train accuracy: 0.93035 test accuracy: 0.9286
train accuracy: 0.94225 test accuracy: 0.9418
train accuracy: 0.9480666666666666 test accuracy: 0.9437
train accuracy: 0.9510166666666666 test accuracy: 0.9454
train accuracy: 0.9607166666666667 test accuracy: 0.9544
train accuracy: 0.9651833333333333 test accuracy: 0.9572
train accuracy: 0.9669833333333333 test accuracy: 0.9582
train accuracy: 0.9674166666666667 test accuracy: 0.9592
train accuracy: 0.97185 test accuracy: 0.9622
train accuracy: 0.9725666666666667 test accuracy: 0.9588
train accuracy: 0.9749333333333333 test accuracy: 0.9648
train accuracy: 0.97725 test accuracy: 0.9654
train accuracy: 0.9757833333333333 test accuracy: 0.9645
train accuracy: 0.978 test accuracy: 0.9653
train accuracy: 0.98165 test accuracy: 0.9674
train accuracy: 0.9811833333333333 test accuracy: 0.9653
train accuracy: 0.9834333333333334 test accuracy: 0.96